In [ ]:
import tensorflow as tf
import keras

In [ ]:
model_resnet = tf.keras.models.load_model('resnet/best/')

In [ ]:
x = model_resnet.layers[-2].output
resnet = keras.Model(model_resnet.input, x)
resnet.summary()

In [ ]:
model_vgg19 = tf.keras.models.load_model('vgg19/180-500-200-adam-mse-binaryacc')
model_vgg19.summary()
y = model_vgg19.layers[-2].output
vgg19 = keras.Model(model_vgg19.input, y)
vgg19.summary()

In [ ]:
model_densenet = tf.keras.models.load_model('denseNet/180-1000-500-200-adam-mse-binaryacc')
model_densenet.summary()
z = model_densenet.layers[-2].output
densenet = keras.Model(model_densenet.input, z)
densenet.summary()

In [ ]:
models = [densenet, vgg19, resnet]

In [ ]:
def ensembleModels(models, model_input):
    # collect outputs of models in a list
    yModels=[model(model_input) for model in models]     # averaging outputs
    yAvg=keras.layers.average(yModels)     # build model from same input and avg output
    modelEns = keras.Model(inputs=model_input, outputs=yAvg,    name='ensemble')  
   
    return modelEns

ensembled_model = ensembleModels(models, vgg19.input)

In [ ]:
x = ensembled_model.layers[-1].output
x = keras.layers.Dense(1)(x)
model = keras.Model(ensembled_model.input, x)
model.summary()

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
             loss=tf.keras.losses.MeanSquaredError(),
             metrics=tf.keras.metrics.BinaryAccuracy())

In [ ]:
path = 'models/ensemble-nofreeze-200'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                                    filepath=path,
                                                    save_weights_only=False,
                                                    monitor='val_loss',
                                                    mode='min',
                                                    save_best_only=True,
                                                    verbose=0)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [ ]:
datapath = "../../../../deeplearning/TensorFlow2.0_ResNet/dataset/"
print(os.path.exists(datapath))
image_dims = (180, 180)
batch_size = 64

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./255)
test_datagen = ImageDataGenerator(rescale = 1./255)
validation_datagen = ImageDataGenerator(rescale = 1./255)
train_generator = train_datagen.flow_from_directory(os.path.join(datapath, "train"),
                                             target_size = image_dims,
                                            batch_size = batch_size,
                                             class_mode = 'binary',
                                            color_mode = 'rgb',
                                                   shuffle=True)
test_generator = test_datagen.flow_from_directory(os.path.join(datapath, "test"), 
                                                             target_size = image_dims,
                                                              batch_size = batch_size,
                                                             class_mode = 'binary', 
                                                             color_mode = 'rgb')
validation_generator = validation_datagen.flow_from_directory(os.path.join(datapath, "valid"), 
                                                     target_size = image_dims,
                                                      batch_size = batch_size,
                                                     class_mode = 'binary', 
                                                     color_mode = 'rgb',
                                                             shuffle=True)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
n = 10
plt.figure(figsize=(20, 4))
xtrain, ytrain = next(train_generator)
for i in range(1, n):
    ax = plt.subplot(1, n, i)
    plt.imshow(xtrain[i])
    ax.title.set_text(ytrain[i])
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

In [ ]:
tf.get_logger().setLevel('ERROR')

In [ ]:
history = model.fit(train_generator,
                    epochs=5,
                    validation_data=validation_generator,
                   callbacks = [model_checkpoint_callback])